In [2]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

# Warning 메세지 무시
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
pd.set_option('mode.chained_assignment', None) # chained_assignment 경고 무시

In [2]:
# Oracle DB 연결
# 접속정보(connection string) : ID/PASS@CONNECTION_ALIAS
# CONNECTION_ALIAS : Oracle TNSNAMES.ORA 파일에 있는 접속정보 별칭(ALIAS)
# conn_ora = cx_Oracle.connect("prep1/prep1@XE")

conn_ora = cx_Oracle.connect("ysu/ysu@localhost:1521/xe")

# DB 커서(Cursor) 선언
cur = conn_ora.cursor()

In [4]:
rawData_org = pd.read_sql("""select 1 gubun
, c_stud_no 학번
, c_stud_nm 성명
, c_gender 성별
, c_dept_nm 소속학과
, c_findjob_type_2 취업준비유형
, c_fj_fund_2 구직기초지식 
, c_fj_ready_2 구직활동준비
, c_fj_info_2 정보탐색활동
, c_fj_spec_2 스펙업활동
, c_fj_apply_2 지원활동
, c_fj_certi_2 자격증
, c_grad_jumsu 학점
, c_fj_english_2 영어
, c_fj_lang_2 기타어학
, c_fj_activity_2 활동수준
, c_fj_rstatus_2 현재준비상태
, c_fj_tech_2 구직기술
, c_fj_docu_2 구직서류작성능력
, c_fj_interview_2 구직면접능력
, c_fj_willing_2 구직의지
, c_fj_goal_2 구직목표명확성
, c_fj_tot_2 종합역량점수
, c_employ_cls 취업유형분류
, decode(c_employ_cls, '미취업자', 1, 0) 미취업여부
from v_student_c_f_j
where c_fj_tot_2 is not null and c_employ_cls is not null and c_grad_date in ('201708', '201802', '201808', '201902', '201908', '202002')
union all
-- 2022.2월 졸업생 미취업 위험률 예측을 위한 데이터셋
select 2 gubun
, c_stud_no 학번
, c_stud_nm 성명
, c_gender 성별
, c_dept_nm 소속학과
, c_findjob_type_2 취업준비유형
, c_fj_fund_2 구직기초지식 
, c_fj_ready_2 구직활동준비
, c_fj_info_2 정보탐색활동
, c_fj_spec_2 스펙업활동
, c_fj_apply_2 지원활동
, c_fj_certi_2 자격증
, c_grad_jumsu 학점
, c_fj_english_2 영어
, c_fj_lang_2 기타어학
, c_fj_activity_2 활동수준
, c_fj_rstatus_2 현재준비상태
, c_fj_tech_2 구직기술
, c_fj_docu_2 구직서류작성능력
, c_fj_interview_2 구직면접능력
, c_fj_willing_2 구직의지
, c_fj_goal_2 구직목표명확성
, c_fj_tot_2 종합역량점수
, NULL 취업유형분류
, NULL 미취업여부
from v_student_c_f_j
where c_fj_tot_2 is not null and c_grad_date is null -- in ('202108', '202202')""", con=conn_ora)

rawData_org

,GUBUN,학번,성명,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,...,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,종합역량점수,취업유형분류,미취업여부
0,1,2012111002,고경오,남자,디지털전자과,중간집단,37.71,37.19,30.90,40.83,...,32.33,42.05,46.87,41.80,51.11,41.14,45.47,41.67,취업자(건보),0.0
1,1,2012111006,김동진,남자,디지털전자과,마음만 취준생,35.34,34.15,32.16,30.42,...,31.19,37.11,38.45,34.22,42.59,67.49,38.26,42.74,미취업자,1.0
2,1,2012111012,김택훈,남자,디지털전자과,중간집단,47.16,39.83,32.16,40.83,...,35.96,43.70,43.26,34.22,52.32,56.64,50.88,47.55,취업자(건보),0.0
3,1,2012111014,도형래,남자,디지털전자과,중간집단,40.07,48.24,49.86,46.04,...,48.68,47.81,44.46,43.96,43.81,41.14,45.47,43.88,미취업자,1.0
4,1,2012111024,위성찬,남자,디지털전자과,중간집단,37.71,41.47,32.16,35.63,...,35.96,46.99,41.46,40.71,41.37,65.94,50.88,47.49,취업자(프리랜서),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,2,2016101815,조영창,남자,사회복지과,중간집단,49.53,43.50,41.01,35.63,...,37.55,49.46,44.46,42.88,45.02,44.24,52.68,46.88,None,NaN
5703,2,2016102137,김동현,남자,관광과 관광중국어전공,중간집단,47.16,36.65,37.22,35.63,...,36.19,37.11,53.48,49.37,55.97,50.44,47.27,47.00,None,NaN
5704,2,2017100899,조윤해,여자,보건의료행정과,취업적신호,35.34,33.13,30.90,30.42,...,29.15,37.11,48.07,47.20,47.46,41.14,29.25,37.39,None,NaN
5705,2,2018100673,안진호,남자,호텔외식조리과 호텔조리전공,준비된 취준생,63.71,44.14,42.91,46.04,...,44.59,43.70,66.71,69.92,59.62,72.15,59.89,61.32,None,NaN


In [5]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(5707, 25)

In [6]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['GUBUN', '성별', '소속학과', '취업준비유형', '구직기초지식', '구직활동준비', '정보탐색활동',
                    '스펙업활동', '지원활동', '자격증', '학점', '영어', '기타어학', '활동수준', '현재준비상태', '구직기술',
                    '구직서류작성능력', '구직면접능력', '구직의지', '구직목표명확성', '미취업여부']

rawData = rawData_org[feature_columns_to_use]
rawData

,GUBUN,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,...,영어,기타어학,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부
0,1,남자,디지털전자과,중간집단,37.71,37.19,30.90,40.83,31.37,52.33,...,39.2,26.15,32.33,42.05,46.87,41.80,51.11,41.14,45.47,0.0
1,1,남자,디지털전자과,마음만 취준생,35.34,34.15,32.16,30.42,35.07,39.00,...,39.2,26.15,31.19,37.11,38.45,34.22,42.59,67.49,38.26,1.0
2,1,남자,디지털전자과,중간집단,47.16,39.83,32.16,40.83,38.78,50.11,...,39.2,26.15,35.96,43.70,43.26,34.22,52.32,56.64,50.88,0.0
3,1,남자,디지털전자과,중간집단,40.07,48.24,49.86,46.04,49.89,61.22,...,39.2,26.15,48.68,47.81,44.46,43.96,43.81,41.14,45.47,1.0
4,1,남자,디지털전자과,중간집단,37.71,41.47,32.16,35.63,42.48,45.67,...,39.2,26.15,35.96,46.99,41.46,40.71,41.37,65.94,50.88,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,2,남자,사회복지과,중간집단,49.53,43.50,41.01,35.63,38.78,72.33,...,39.2,26.15,37.55,49.46,44.46,42.88,45.02,44.24,52.68,NaN
5703,2,남자,관광과 관광중국어전공,중간집단,47.16,36.65,37.22,35.63,38.78,39.00,...,39.2,26.15,36.19,37.11,53.48,49.37,55.97,50.44,47.27,NaN
5704,2,여자,보건의료행정과,취업적신호,35.34,33.13,30.90,30.42,31.37,39.00,...,39.2,26.15,29.15,37.11,48.07,47.20,47.46,41.14,29.25,NaN
5705,2,남자,호텔외식조리과 호텔조리전공,준비된 취준생,63.71,44.14,42.91,46.04,46.19,50.11,...,39.2,26.15,44.59,43.70,66.71,69.92,59.62,72.15,59.89,NaN


In [7]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별','소속학과','취업준비유형']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData[feature] = le.fit_transform(rawData[feature])   

<ipython-input-7-d4225ce65ce4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawData[feature] = le.fit_transform(rawData[feature])


In [8]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
LearnData = rawData.loc[rawData.GUBUN==1, : ]

In [9]:
X = LearnData.loc[:, feature_columns_to_use[1:-1]]  
y = LearnData.loc[:, '미취업여부']  # 분류 클래스(class)

from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
# imblearn(imbalanced data 문제해결을 위한 패키지) import
from imblearn.over_sampling import SMOTE

# SMOTE 모델 설정
smote = SMOTE(random_state=0)

# SMOTE 기법에 의한 Oversampling 진행
X_over,y_over = smote.fit_sample(X,y)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X.shape, y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_over.shape, y_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_over).value_counts())

# Oversampling dataset에 대한 training set과 test set 구분 (3:1)
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over, test_size=0.3, random_state=0)

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (5530, 19) (5530,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (8210, 19) (8210,)
SMOTE 적용 후 레이블 값 분포: 
 1.0    4105
0.0    4105
Name: 미취업여부, dtype: int64


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 모델링 및 학습/검증
def f_model_fit(cls, model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    f_metrics(cls, y_test,pred)
    
# 분류 성능평가 지표
def f_metrics(cls, y_test,pred):
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    print(cls + '정확도 : {0:.3f}, 정밀도 : {1:.3f}, 재현율 : {2:.3f}, f1-score : {3:.3f}'.format(accuracy, precision, recall, f1))

In [12]:
# 분석기법 : Decision_tree Classification
from sklearn.tree import DecisionTreeClassifier  # 의사결정트리 기법에 관련된 모듈

decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
f_model_fit('Decision_tree(SMOTE적용 전) -> ', decision_tree, X_train, X_test, y_train, y_test)
f_model_fit('Decision_tree(SMOTE적용 후) -> ', decision_tree, X_train_over, X_test_over, y_train_over, y_test_over)

Decision_tree(SMOTE적용 전) -> 정확도 : 0.727, 정밀도 : 0.355, 재현율 : 0.025, f1-score : 0.046
Decision_tree(SMOTE적용 후) -> 정확도 : 0.608, 정밀도 : 0.594, 재현율 : 0.705, f1-score : 0.645


In [13]:
# 분석기법 : Random_forest Classification
from sklearn.ensemble import RandomForestClassifier  # 랜덤포리스트 기법에 관련된 모듈

random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
f_model_fit('Random_forest(SMOTE적용 전) -> ', random_forest, X_train, X_test, y_train, y_test)
f_model_fit('Random_forest(SMOTE적용 후) -> ', random_forest, X_train_over, X_test_over, y_train_over, y_test_over)

Random_forest(SMOTE적용 전) -> 정확도 : 0.724, 정밀도 : 0.410, 재현율 : 0.072, f1-score : 0.123
Random_forest(SMOTE적용 후) -> 정확도 : 0.796, 정밀도 : 0.842, 재현율 : 0.734, f1-score : 0.785


In [14]:
# 분석기법 : Xgboost Classification
import xgboost as xgb # xgboost 기법에 관련된 모듈

gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05, eval_metric='logloss')
f_model_fit('XGBoost(SMOTE적용 전) -> ', gbm, X_train, X_test, y_train, y_test)
f_model_fit('XGBoost(SMOTE적용 후) -> ', gbm, X_train_over, X_test_over, y_train_over, y_test_over)

C:\Users\a\anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBoost(SMOTE적용 전) -> 정확도 : 0.709, 정밀도 : 0.371, 재현율 : 0.126, f1-score : 0.188
XGBoost(SMOTE적용 후) -> 정확도 : 0.822, 정밀도 : 0.898, 재현율 : 0.730, f1-score : 0.805


In [15]:
# 분석기법 : LightGBM Classification
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
f_model_fit('Lightgbm(SMOTE적용 전) -> ', lgb, X_train, X_test, y_train, y_test)
f_model_fit('Lightgbm(SMOTE적용 후) -> ', lgb, X_train_over, X_test_over, y_train_over, y_test_over)

Lightgbm(SMOTE적용 전) -> 정확도 : 0.689, 정밀도 : 0.316, 재현율 : 0.140, f1-score : 0.194
Lightgbm(SMOTE적용 후) -> 정확도 : 0.819, 정밀도 : 0.866, 재현율 : 0.759, f1-score : 0.809


In [16]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature_dtree = pd.DataFrame({'중요도' : decision_tree.feature_importances_}, index = feature_columns_to_use[1:-1])
sel_feature_rforest = pd.DataFrame({'중요도' : random_forest.feature_importances_}, index = feature_columns_to_use[1:-1])
sel_feature_xgboost = pd.DataFrame({'중요도' : gbm.feature_importances_}, index = feature_columns_to_use[1:-1])
sel_feature_lgbm = pd.DataFrame({'중요도' : lgb.feature_importances_}, index = feature_columns_to_use[1:-1])

In [17]:
# 중요도의 내림차순으로 정렬
sel_feature_dtree.sort_values(by='중요도', ascending=False)

,중요도
구직목표명확성,0.460623
학점,0.207672
스펙업활동,0.111128
구직의지,0.066915
성별,0.065201
소속학과,0.062952
정보탐색활동,0.025509
활동수준,0.000000
구직면접능력,0.000000
구직서류작성능력,0.000000


In [18]:
# 중요도의 내림차순으로 정렬
sel_feature_rforest.sort_values(by='중요도', ascending=False)

,중요도
학점,0.087958
구직목표명확성,0.074676
구직기초지식,0.069766
구직의지,0.068790
현재준비상태,0.067717
구직활동준비,0.065289
구직면접능력,0.064089
소속학과,0.063878
구직서류작성능력,0.063440
정보탐색활동,0.061485


In [19]:
# 중요도의 내림차순으로 정렬
sel_feature_xgboost.sort_values(by='중요도', ascending=False)

,중요도
스펙업활동,0.144342
지원활동,0.098963
구직기초지식,0.087274
구직목표명확성,0.083740
현재준비상태,0.080000
성별,0.064492
자격증,0.057819
구직의지,0.052410
학점,0.043050
소속학과,0.041486


In [20]:
# 중요도의 내림차순으로 정렬
sel_feature_lgbm.sort_values(by='중요도', ascending=False)

,중요도
학점,7256
구직활동준비,5243
구직의지,5037
소속학과,4849
구직목표명확성,4612
활동수준,4467
정보탐색활동,4404
구직서류작성능력,4249
구직기초지식,4031
구직기술,3967


In [21]:
# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr_dtree = decision_tree.predict(X_test)
y_pred_tr_rforest = random_forest.predict(X_test)
y_pred_tr_xgboost = gbm.predict(X_test)
y_pred_tr_lgbm = lgb.predict(X_test)

# predict_proba() : 각 input 개체에 대한 예측클래스 확률 구함
y_pred_tr_prob_dtree = decision_tree.predict_proba(X_test)
y_pred_tr_prob_rforest = random_forest.predict_proba(X_test)
y_pred_tr_prob_xgboost = gbm.predict_proba(X_test)
y_pred_tr_prob_lgbm = lgb.predict_proba(X_test)

In [22]:
X_test_df_dtree = pd.DataFrame(X_test, copy=True)
X_test_df_rforest = pd.DataFrame(X_test, copy=True)
X_test_df_xgboost = pd.DataFrame(X_test, copy=True)
X_test_df_lgbm = pd.DataFrame(X_test, copy=True)

In [23]:
X_test_df_dtree['미취업여부'] = y_test
X_test_df_rforest['미취업여부'] = y_test
X_test_df_xgboost['미취업여부'] = y_test
X_test_df_lgbm['미취업여부'] = y_test

X_test_df_dtree['미취업여부예측'] = y_pred_tr_dtree
X_test_df_rforest['미취업여부예측'] = y_pred_tr_rforest
X_test_df_xgboost['미취업여부예측'] = y_pred_tr_xgboost
X_test_df_lgbm['미취업여부예측'] = y_pred_tr_lgbm

X_test_df_dtree['미취업확률'] = y_pred_tr_prob_dtree[:,1]
X_test_df_rforest['미취업확률'] = y_pred_tr_prob_rforest[:,1]
X_test_df_xgboost['미취업확률'] = y_pred_tr_prob_xgboost[:,1]
X_test_df_lgbm['미취업확률'] = y_pred_tr_prob_lgbm[:,1]

In [24]:
X_test_df_dtree

,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,학점,...,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부,미취업여부예측,미취업확률
15,0,28,3,49.53,42.91,51.76,35.63,46.19,50.11,2.07,...,44.59,41.23,40.86,36.39,45.02,67.49,49.07,0.0,1.0,0.512101
4616,1,11,3,54.26,49.11,47.33,35.63,46.19,87.89,3.27,...,43.00,55.22,58.89,59.10,55.97,48.89,56.28,0.0,1.0,0.512101
3559,1,3,3,56.62,43.08,39.12,46.04,42.48,39.00,3.86,...,41.64,44.52,50.47,52.61,46.24,51.99,58.08,1.0,0.0,0.478477
3552,0,6,3,56.62,61.81,57.45,66.88,53.59,99.00,3.36,...,59.35,64.27,52.88,59.10,43.81,44.24,56.28,0.0,1.0,0.512101
3009,0,13,3,58.98,56.04,59.97,66.88,61.00,56.78,3.93,...,63.44,48.63,48.07,48.29,46.24,59.74,59.89,0.0,0.0,0.413174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,1,0,3,68.44,46.87,58.71,51.25,57.30,39.00,3.18,...,56.63,37.11,45.06,45.04,43.81,41.14,43.67,0.0,0.0,0.313559
760,1,8,3,40.07,43.52,38.48,35.63,31.37,70.11,2.89,...,33.46,53.57,44.46,43.96,43.81,41.14,41.87,0.0,1.0,0.661017
3496,0,6,3,63.71,50.88,42.28,56.46,57.30,72.33,2.73,...,52.31,49.46,45.06,45.04,43.81,41.14,45.47,0.0,0.0,0.375510
1031,0,29,3,40.07,40.92,35.95,35.63,42.48,39.00,3.50,...,37.32,44.52,39.05,33.14,45.02,59.74,49.07,1.0,1.0,0.554455


In [25]:
X_test_df_rforest

,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,학점,...,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부,미취업여부예측,미취업확률
15,0,28,3,49.53,42.91,51.76,35.63,46.19,50.11,2.07,...,44.59,41.23,40.86,36.39,45.02,67.49,49.07,0.0,0.0,0.45
4616,1,11,3,54.26,49.11,47.33,35.63,46.19,87.89,3.27,...,43.00,55.22,58.89,59.10,55.97,48.89,56.28,0.0,0.0,0.10
3559,1,3,3,56.62,43.08,39.12,46.04,42.48,39.00,3.86,...,41.64,44.52,50.47,52.61,46.24,51.99,58.08,1.0,1.0,0.70
3552,0,6,3,56.62,61.81,57.45,66.88,53.59,99.00,3.36,...,59.35,64.27,52.88,59.10,43.81,44.24,56.28,0.0,0.0,0.10
3009,0,13,3,58.98,56.04,59.97,66.88,61.00,56.78,3.93,...,63.44,48.63,48.07,48.29,46.24,59.74,59.89,0.0,0.0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,1,0,3,68.44,46.87,58.71,51.25,57.30,39.00,3.18,...,56.63,37.11,45.06,45.04,43.81,41.14,43.67,0.0,0.0,0.15
760,1,8,3,40.07,43.52,38.48,35.63,31.37,70.11,2.89,...,33.46,53.57,44.46,43.96,43.81,41.14,41.87,0.0,0.0,0.00
3496,0,6,3,63.71,50.88,42.28,56.46,57.30,72.33,2.73,...,52.31,49.46,45.06,45.04,43.81,41.14,45.47,0.0,0.0,0.00
1031,0,29,3,40.07,40.92,35.95,35.63,42.48,39.00,3.50,...,37.32,44.52,39.05,33.14,45.02,59.74,49.07,1.0,1.0,0.65


In [26]:
X_test_df_xgboost

,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,학점,...,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부,미취업여부예측,미취업확률
15,0,28,3,49.53,42.91,51.76,35.63,46.19,50.11,2.07,...,44.59,41.23,40.86,36.39,45.02,67.49,49.07,0.0,0.0,0.410053
4616,1,11,3,54.26,49.11,47.33,35.63,46.19,87.89,3.27,...,43.00,55.22,58.89,59.10,55.97,48.89,56.28,0.0,0.0,0.164838
3559,1,3,3,56.62,43.08,39.12,46.04,42.48,39.00,3.86,...,41.64,44.52,50.47,52.61,46.24,51.99,58.08,1.0,0.0,0.395289
3552,0,6,3,56.62,61.81,57.45,66.88,53.59,99.00,3.36,...,59.35,64.27,52.88,59.10,43.81,44.24,56.28,0.0,0.0,0.060685
3009,0,13,3,58.98,56.04,59.97,66.88,61.00,56.78,3.93,...,63.44,48.63,48.07,48.29,46.24,59.74,59.89,0.0,0.0,0.273421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,1,0,3,68.44,46.87,58.71,51.25,57.30,39.00,3.18,...,56.63,37.11,45.06,45.04,43.81,41.14,43.67,0.0,0.0,0.042754
760,1,8,3,40.07,43.52,38.48,35.63,31.37,70.11,2.89,...,33.46,53.57,44.46,43.96,43.81,41.14,41.87,0.0,0.0,0.355832
3496,0,6,3,63.71,50.88,42.28,56.46,57.30,72.33,2.73,...,52.31,49.46,45.06,45.04,43.81,41.14,45.47,0.0,0.0,0.188639
1031,0,29,3,40.07,40.92,35.95,35.63,42.48,39.00,3.50,...,37.32,44.52,39.05,33.14,45.02,59.74,49.07,1.0,1.0,0.512361


In [27]:
X_test_df_lgbm

,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,학점,...,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부,미취업여부예측,미취업확률
15,0,28,3,49.53,42.91,51.76,35.63,46.19,50.11,2.07,...,44.59,41.23,40.86,36.39,45.02,67.49,49.07,0.0,0.0,3.372594e-02
4616,1,11,3,54.26,49.11,47.33,35.63,46.19,87.89,3.27,...,43.00,55.22,58.89,59.10,55.97,48.89,56.28,0.0,0.0,6.057760e-06
3559,1,3,3,56.62,43.08,39.12,46.04,42.48,39.00,3.86,...,41.64,44.52,50.47,52.61,46.24,51.99,58.08,1.0,1.0,9.998519e-01
3552,0,6,3,56.62,61.81,57.45,66.88,53.59,99.00,3.36,...,59.35,64.27,52.88,59.10,43.81,44.24,56.28,0.0,0.0,6.537914e-07
3009,0,13,3,58.98,56.04,59.97,66.88,61.00,56.78,3.93,...,63.44,48.63,48.07,48.29,46.24,59.74,59.89,0.0,0.0,2.152773e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,1,0,3,68.44,46.87,58.71,51.25,57.30,39.00,3.18,...,56.63,37.11,45.06,45.04,43.81,41.14,43.67,0.0,0.0,6.919264e-09
760,1,8,3,40.07,43.52,38.48,35.63,31.37,70.11,2.89,...,33.46,53.57,44.46,43.96,43.81,41.14,41.87,0.0,0.0,9.077899e-05
3496,0,6,3,63.71,50.88,42.28,56.46,57.30,72.33,2.73,...,52.31,49.46,45.06,45.04,43.81,41.14,45.47,0.0,0.0,2.804185e-05
1031,0,29,3,40.07,40.92,35.95,35.63,42.48,39.00,3.50,...,37.32,44.52,39.05,33.14,45.02,59.74,49.07,1.0,1.0,9.998603e-01


In [28]:
X_test_df_dtree.to_excel('미취업확률분석(decision_tree)_2022.xlsx')
X_test_df_rforest.to_excel('미취업확률분석(random_forest)_2022.xlsx')
X_test_df_xgboost.to_excel('미취업확률분석(xgboost)_2022.xlsx')
X_test_df_lgbm.to_excel('미취업확률분석(lgbm)_2022.xlsx')

In [29]:
# 2021.08 & 2022.02월 졸업생 미취업 위험률 예측
Graduate22_df = rawData.loc[rawData.GUBUN==2, '성별':'구직목표명확성']

Graduate22_df

,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,학점,영어,기타어학,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성
5530,0,28,3,51.89,42.84,39.75,40.83,38.78,65.67,3.20,39.2,26.15,38.68,46.99,48.67,47.20,48.67,44.24,50.88
5531,0,28,3,40.07,35.17,37.85,35.63,31.37,39.00,3.05,39.2,26.15,33.23,37.11,52.28,50.45,52.32,55.09,47.27
5532,0,28,3,44.80,43.74,35.95,46.04,42.48,39.00,3.87,39.2,26.15,40.50,46.99,65.50,67.76,59.62,41.14,47.27
5533,0,27,3,40.07,37.64,33.43,30.42,38.78,39.00,3.13,39.2,26.15,33.23,42.05,60.69,56.94,62.06,50.44,49.07
5534,0,27,3,35.34,35.60,30.90,30.42,31.37,39.00,3.52,39.2,26.15,29.15,42.05,53.48,50.45,54.76,55.09,49.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,0,12,3,49.53,43.50,41.01,35.63,38.78,72.33,1.90,39.2,26.15,37.55,49.46,44.46,42.88,45.02,44.24,52.68
5703,0,4,3,47.16,36.65,37.22,35.63,38.78,39.00,1.86,39.2,26.15,36.19,37.11,53.48,49.37,55.97,50.44,47.27
5704,1,8,4,35.34,33.13,30.90,30.42,31.37,39.00,2.80,39.2,26.15,29.15,37.11,48.07,47.20,47.46,41.14,29.25
5705,0,34,1,63.71,44.14,42.91,46.04,46.19,50.11,3.40,39.2,26.15,44.59,43.70,66.71,69.92,59.62,72.15,59.89


In [30]:
# 원본데이터의 튜플 수와 컬럼수 출력
Graduate22_df.shape

(177, 19)

In [31]:
# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr_dtree = decision_tree.predict(Graduate22_df)
y_pred_tr_rforest = random_forest.predict(Graduate22_df)
y_pred_tr_xgboost = gbm.predict(Graduate22_df)
y_pred_tr_lgbm = lgb.predict(Graduate22_df)

# predict_proba() : 각 input 개체에 대한 예측클래스 확률 구함
y_pred_tr_prob_dtree = decision_tree.predict_proba(Graduate22_df)
y_pred_tr_prob_rforest = random_forest.predict_proba(Graduate22_df)
y_pred_tr_prob_xgboost = gbm.predict_proba(Graduate22_df)
y_pred_tr_prob_lgbm = lgb.predict_proba(Graduate22_df)

Graduate22_df['미취업여부예측(dtree)'] = y_pred_tr_dtree
Graduate22_df['미취업확률(dtree)'] = y_pred_tr_prob_dtree[:,1]

Graduate22_df['미취업여부예측(rforest)'] = y_pred_tr_rforest
Graduate22_df['미취업확률(rforest)'] = y_pred_tr_prob_rforest[:,1]

Graduate22_df['미취업여부예측(xgboost)'] = y_pred_tr_xgboost
Graduate22_df['미취업확률(xgboost)'] = y_pred_tr_prob_xgboost[:,1]

Graduate22_df['미취업여부예측(lgbm)'] = y_pred_tr_lgbm
Graduate22_df['미취업확률(lgbm)'] = y_pred_tr_prob_lgbm[:,1]

Graduate22_df['학번'] = rawData_org.loc[rawData_org.GUBUN==2, '학번']
Graduate22_df['성명'] = rawData_org.loc[rawData_org.GUBUN==2, '성명']
Graduate22_df['성별'] = rawData_org.loc[rawData_org.GUBUN==2, '성별']
Graduate22_df['소속학과'] = rawData_org.loc[rawData_org.GUBUN==2, '소속학과']
Graduate22_df['취업준비유형'] = rawData_org.loc[rawData_org.GUBUN==2, '취업준비유형']
Graduate22_df['종합역량점수'] = rawData_org.loc[rawData_org.GUBUN==2, '종합역량점수']

Graduate22_df

,성별,소속학과,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,학점,...,미취업확률(dtree),미취업여부예측(rforest),미취업확률(rforest),미취업여부예측(xgboost),미취업확률(xgboost),미취업여부예측(lgbm),미취업확률(lgbm),학번,성명,종합역량점수
5530,남자,정보통신과,중간집단,51.89,42.84,39.75,40.83,38.78,65.67,3.20,...,0.512101,1.0,0.60,1.0,0.557146,1.0,9.986544e-01,2016100131,조현재,47.70
5531,남자,정보통신과,중간집단,40.07,35.17,37.85,35.63,31.37,39.00,3.05,...,0.512101,0.0,0.35,0.0,0.309608,0.0,1.915424e-04,2016100147,김형찬,45.98
5532,남자,정보통신과,중간집단,44.80,43.74,35.95,46.04,42.48,39.00,3.87,...,0.413174,0.0,0.45,0.0,0.449574,0.0,3.550609e-01,2016100156,박성진,48.49
5533,남자,전기과,중간집단,40.07,37.64,33.43,30.42,38.78,39.00,3.13,...,0.512101,0.0,0.45,0.0,0.234590,0.0,5.285185e-04,2016100172,최승근,47.58
5534,남자,전기과,중간집단,35.34,35.60,30.90,30.42,31.37,39.00,3.52,...,0.554455,0.0,0.45,0.0,0.386820,0.0,2.319504e-02,2016100178,박병찬,45.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,남자,사회복지과,중간집단,49.53,43.50,41.01,35.63,38.78,72.33,1.90,...,0.512101,0.0,0.50,1.0,0.604071,0.0,9.215950e-02,2016101815,조영창,46.88
5703,남자,관광과 관광중국어전공,중간집단,47.16,36.65,37.22,35.63,38.78,39.00,1.86,...,0.512101,0.0,0.35,1.0,0.549605,0.0,1.858053e-02,2016102137,김동현,47.00
5704,여자,보건의료행정과,취업적신호,35.34,33.13,30.90,30.42,31.37,39.00,2.80,...,0.661017,0.0,0.40,1.0,0.615290,1.0,5.540831e-01,2017100899,조윤해,37.39
5705,남자,호텔외식조리과 호텔조리전공,준비된 취준생,63.71,44.14,42.91,46.04,46.19,50.11,3.40,...,0.103448,0.0,0.20,0.0,0.238449,0.0,4.532990e-05,2018100673,안진호,61.32


In [32]:
Graduate22_df.to_excel('미취업확률분석(22년졸업생)_2022.xlsx')